<a href="https://colab.research.google.com/github/darrelbelvin/AutoCAD-LISP/blob/main/cohort24/Model%20Team%201%20(Haystack%20Framework)/haystack_GQA_using_embeddingRe_plus_ranker_served_to_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Wed Aug 10 21:53:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install --upgrade pip
!pip install farm-haystack[faiss]

In [3]:
from haystack.nodes import Seq2SeqGenerator, SentenceTransformersRanker, EmbeddingRetriever, DensePassageRetriever, PreProcessor
from haystack.utils import  convert_files_to_docs, print_answers
from haystack.document_stores import FAISSDocumentStore
from haystack import Pipeline
import logging, sys

In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", embedding_dim=128)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [5]:
%%capture
!git clone https://github.com/christsakm/conv-ai/

In [6]:
logging.getLogger("haystack").setLevel(logging.WARNING) # stops this cell output from filling the screen
data_dir = 'conv-ai/data/'
docs = convert_files_to_docs(dir_path=data_dir)

In [7]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    remove_substrings=['\n', '\\'],
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=False,
    split_overlap = 15
)
docs2 = preprocessor.process(docs)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 2058/2058 [00:01<00:00, 1142.71docs/s]


In [8]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="yjernite/retribert-base-uncased",
)

Downloading:   0%|          | 0.00/487 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/310M [00:00<?, ?B/s]

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

Downloading:   0%|          | 0.00/791 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
generator = Seq2SeqGenerator(
    model_name_or_path="vblagoje/bart_lfqa", 
    num_beams=8,
    min_length=50,
    max_length=256
)

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [11]:
document_store.delete_documents()
document_store.write_documents(docs2)
document_store.get_document_count()
document_store.update_embeddings(retriever)

Writing Documents:   0%|          | 0/19950 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/19680 [00:00<?, ? docs/s]

Creating Embeddings:   0%|          | 0/313 [00:00<?, ? Batches/s]

Creating Embeddings:   0%|          | 0/303 [00:00<?, ? Batches/s]

In [12]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=ranker, name="Ranker", inputs=["Retriever"])
pipe.add_node(component=generator, name="Generator", inputs=["Ranker"])

## Serving to Telegram
While last cell is running bot can be interacted with at [t.me/Fellows_Convo_AI_Bot](https://t.me/Fellows_Convo_AI_Bot)

In [13]:
%%capture
%pip install aiogram
%pip install nest-asyncio

In [14]:
API_TOKEN = '5539304790:AAFWJLskyJCwEyXCyIURs_IFKqcFQLS11bw'
# This token controls t.me/Fellows_Convo_AI_Bot

In [15]:
from aiogram import Bot, Dispatcher, executor, types
import nest_asyncio
nest_asyncio.apply()

In [16]:
bot = Bot(token=API_TOKEN)
dp = Dispatcher(bot)

In [17]:
@dp.message_handler(commands=['start', 'help'])
async def send_welcome(message: types.Message):
    """
    This handler will be called when user sends `/start` or `/help` command
    """
    await message.reply("Hi!\nI'm a conversational AI!\nI can answer questions about nutrition.")

In [18]:
@dp.message_handler()
async def echo(message: types.Message):
    print(message)
    prediction = pipe.run(
      query=message.text,params={"Retriever": {"top_k": 50}, "Ranker": {"top_k": 4}})
    answer = prediction["answers"][0].answer
    print_answers(prediction, details="minimum")
    await message.answer(answer)

In [20]:
sys.stdout = open('queries.log', 'a') # redirects output from print_answers to a file
executor.start_polling(dp, skip_updates=True)
# errors will be thrown if more than one notebook is running this at the same time

In [19]:
sys.stdout = sys.__stdout__ # reset standard out